In [ ]:
!pip install pyarrow duckdb pandas numpy -q

# 📦 Instalação de Pacotes

Antes de começar, vamos instalar os pacotes necessários:

# Capitulo 07 Ipc Serializacao

Notebook gerado automaticamente a partir do código fonte python.


## 📚 Importação de Bibliotecas

Importando as bibliotecas necessárias para o capítulo:

In [ ]:
import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np
import time
import os

print("Bibliotecas carregadas!")


## 🔧 Configuração Inicial

Preparando dados de exemplo e conexão com DuckDB:

In [ ]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    # Aumentando para 100k para ver diferença na compressão
    data = pa.table({
        'id': range(100_000),
        'valor': np.random.randn(100_000),
        'categoria': np.random.choice(['A', 'B', 'C', 'D', 'E'], 100_000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()

## 1️⃣ IPC Format

Inter-Process Communication Format do Apache Arrow:

In [ ]:
print(f"\n--- {'IPC Format'.upper()} ---")

# 7.1.1 Introdução ao IPC (Inter-Process Communication)
# O formato IPC é uma representação binária de Record Batches
# Ele pode ser escrito em streams (soquetes) ou arquivos.

# 7.1.2 Escrevendo em um stream IPCInMemory
sink = pa.BufferOutputStream()
with pa.ipc.new_stream(sink, data.schema) as writer:
    # Escrever os batches da nossa tabela
    for batch in data.to_batches():
        writer.write_batch(batch)

buffer = sink.getvalue()
print(f"Stream IPC criado na memória.")
print(f"  Tamanho do buffer: {buffer.size / 1024:.2f} KB")

# 7.1.3 Lendo a partir de um stream IPC
stream_reader = pa.ipc.open_stream(buffer)
recovered_table = stream_reader.read_all()

print(f"\nTabela recuperada do stream IPC:")
print(f"  Linhas: {recovered_table.num_rows}")
print(f"  Schema: {recovered_table.schema.names}")

## 2️⃣ Feather Format

Formato Feather para armazenamento eficiente:

In [ ]:
print(f"\n--- {'Feather format'.upper()} ---")

# 7.2.1 O que é o Feather?
# Feather (V2) é basicamente o formato IPC do Arrow persistido em disco.
# É extremamente rápido porque é idêntico ao layout na RAM.

import pyarrow.feather as feather

# 7.2.2 Escrevendo arquivo Feather
file_path = "dados_cap7.feather"
feather.write_feather(data, file_path)
print(f"Arquivo Feather salvo: {file_path}")

# 7.2.3 Lendo arquivo Feather
# Podemos ler como Table ou como Pandas
table_feather = feather.read_table(file_path)
print(f"Leitura bem-sucedida (Table): {table_feather.num_rows} linhas")

# Limpeza
if os.path.exists(file_path):
    os.remove(file_path)
    print("Arquivo temporário removido.")

## 3️⃣ Serialização para Disco

Salvando e carregando dados do disco:

In [ ]:
print(f"\n--- {'Serialização para disco'.upper()} ---")

# 7.3.1 Serialização Genérica
# Métodos antigos como array.serialize() foram removidos.
# A recomendação atual é usar o formato IPC mesmo para componentes individuais.

# Serializando um Array (via RecordBatch)
array = pa.array([10, 20, 30, 40])
batch = pa.RecordBatch.from_arrays([array], names=['v'])

sink = pa.BufferOutputStream()
with pa.ipc.new_stream(sink, batch.schema) as writer:
    writer.write_batch(batch)

buffer_array = sink.getvalue()
print(f"Array serializado para buffer IPC: {buffer_array.size} bytes")

# 7.3.2 Exemplo usando LocalFileSystem
from pyarrow import fs
local = fs.LocalFileSystem()

with local.open_output_stream("raw_batches.arrow") as stream:
    with pa.ipc.new_file(stream, data.schema) as writer:
        writer.write_table(data)

info = local.get_file_info("raw_batches.arrow")
print(f"Arquivo IPC (file format) criado: {info.size / 1024:.2f} KB")

# Limpeza
if os.path.exists("raw_batches.arrow"):
    os.remove("raw_batches.arrow")

## 4️⃣ Shared Memory

Compartilhamento de dados em memória compartilhada:

In [ ]:
print(f"\n--- {'Shared memory'.upper()} ---")

# 7.4.1 Mapas de Memória (Memory-Mapped Files)
# Permite que múltiplos processos acessem o mesmo arquivo como se estivesse na RAM,
# sem copiar os dados para o espaço de endereçamento de cada processo.

import pyarrow.feather as feather

# 1. Criar um arquivo Feather formatado
feather.write_feather(data, "shared_data.feather")

# 2. Abrir usando memory map (mmap)
# mmap=True é o padrão no read_table para feather, mas vamos explicitar
mmap_table = feather.read_table("shared_data.feather", memory_map=True)

print("Tabela mapeada em memória (Shared Memory Simulation):")
print(f"  Referência de memória: {mmap_table[0].chunk(0).buffers()[1]}")
print("Os dados permanecem no disco/cache do SO e são acessados sob demanda.")

# Limpeza
os.remove("shared_data.feather")

## 5️⃣ Compressão

Técnicas de compressão de dados:

In [ ]:
print(f"--- {'Compressão'.upper()} ---")

# 7.5.1 Algoritmos de Compressão Suportados
# Arrow suporta LZ4 e ZSTD nativamente no formato IPC/Feather.

def benchmark_compression(compression):
    sink = pa.BufferOutputStream()
    options = pa.ipc.IpcWriteOptions(compression=compression)
    
    start = time.perf_counter()
    with pa.ipc.new_file(sink, data.schema, options=options) as writer:
        writer.write_table(data)
    end = time.perf_counter()
    
    buf = sink.getvalue()
    return buf.size, end - start

# Comparando
print("Comparativo de Compressão (IPC File):")
print("-" * 40)

for comp in [None, 'lz4', 'zstd']:
    size, duration = benchmark_compression(comp)
    name = comp if comp else "Sem compressão"
    print(f"{name:15}: {size/1024:>9.2f} KB | Tempo: {duration*1000:7.4f}ms")

# Nota: LZ4 geralmente é mais rápido, ZSTD comprime mais (dependendo dos dados).